# traitement des données scrapées

1. Concatener les csv en une seule DataFrame
2. Supprimer les lien_produit qui se doublent 
3. Création des colonnes à partir de la colonne Description 
4. Création de la colonne saveur

In [1]:
import requests
import pandas as pd
import csv
import os
import re

tous les csv scrappés

In [2]:
df_fruit = pd.read_csv("donnees_scrappes/produits_fruit_A&L.csv")
df_dessert = pd.read_csv("donnees_scrappes/produits_dessert_A&L.csv")
df_menthe = pd.read_csv("donnees_scrappes/produits_menthe_A&L.csv")
df_frais = pd.read_csv("donnees_scrappes/produits_frais_A&L.csv")
df_boisson = pd.read_csv("donnees_scrappes/produits_boisson_A&L.csv")
df_classique = pd.read_csv("donnees_scrappes/produits_classique_A&L.csv")

In [3]:
# Ajouter la colonne 'gout' avec la valeur 'fruit' pour chaque ligne
df_fruit['gout'] = 'fruit'
# Ajouter la colonne 'gout' avec la valeur 'dessert' pour chaque ligne
df_dessert['gout'] = 'dessert'
# Ajouter la colonne 'gout' avec la valeur 'menthe' pour chaque ligne
df_menthe['gout'] = 'menthe'
# Ajouter la colonne 'gout' avec la valeur 'frais' pour chaque ligne
df_frais['gout'] = 'frais'
# Ajouter la colonne 'gout' avec la valeur 'boisson' pour chaque ligne
df_boisson['gout'] = 'boisson'
# Ajouter la colonne 'gout' avec la valeur 'classique' pour chaque ligne
df_classique['gout'] = 'classique'

### 1. Concaténer les DataFrames

In [4]:
df_concat = pd.concat([df_fruit, df_dessert, df_menthe, df_frais, df_boisson, df_classique], ignore_index=True)

nb nom_produit / lien_produit unique
1533 de 2332 lignes 

In [5]:
df_concat.shape

(2332, 8)

In [6]:
df_concat['lien_produit'].nunique()

1533

In [7]:
df_concat['Title'].nunique()

1533

### 2. Supprimer les lien_produit qui se doublent 

Pas besoin de suprimer les noms non unique car ca peut etre même type de produit mais different contenance 
- ! mais je suprime les lien du produit qui se doublent 
Créer une nouvelle DataFrame avec des lignes ayant des valeurs uniques dans 'lien_produit'

In [8]:
df_unique_lien = df_concat.drop_duplicates(subset='lien_produit').copy()
df_unique_lien.shape

(1533, 8)

### 3. Création des colonnes à partir de la colonne Description 

de col Features > cols : Contenance	Taux PG/VG	Origine	Frais / Mentholé	Surboosté en arômes	Saveur

In [9]:
import ast  # Pour convertir la chaîne en dictionnaire
# Fonction pour extraire la valeur d'une clé dans la colonne 'Features'
def extract_feature(features_str, feature_name):
    try:
        # Convertir la chaîne en dictionnaire
        features = ast.literal_eval(features_str)
        return features.get(feature_name, None)  # Retourne la valeur ou None si la clé n'existe pas
    except Exception as e:
        print(f"Erreur lors de l'extraction de {feature_name}: {e}")
        return None

# Liste des caractéristiques que vous voulez extraire
features_list = ['Contenance', 'Taux PG/VG', 'Origine', 'Frais / Mentholé', 'Surboosté en arômes', 'Saveur']

# Extraire les caractéristiques dans des colonnes séparées
for feature in features_list:
    df_unique_lien[feature] = df_unique_lien['Features'].apply(lambda x: extract_feature(x, feature))

structuration de la df

In [10]:
# Fonction pour extraire les paragraphes après un <h2> ou <h3> spécifique
def extraire_paragraphes_multiples(description, sections_cibles):
    # Diviser la description en lignes
    lignes = description.splitlines()
    
    # Dictionnaire pour stocker les résultats
    resultats = {section: [] for section in sections_cibles}
    capture = None  # Variable pour indiquer quelle section est en cours de capture

    for ligne in lignes:
        # Si la ligne contient un <h2> ou <h3> avec un texte cible, commencer à capturer
        for section in sections_cibles:
            if ("<h2>" in ligne or "<h3>" in ligne) and section in ligne:
                capture = section
                break  # Pas besoin de vérifier d'autres sections si une correspondance est trouvée
        
        # Si une capture est en cours, ajouter les paragraphes
        if capture and "<p>" in ligne and "</p>" in ligne:
            paragraphe = ligne.replace("<p>", "").replace("</p>", "").strip()
            resultats[capture].append(paragraphe)
        
        # Arrêter la capture à la prochaine balise <h2> ou <h3> non ciblée
        if capture and ("<h2>" in ligne or "<h3>" in ligne) and all(section not in ligne for section in sections_cibles):
            capture = None

    # Joindre les paragraphes de chaque section
    return {section: "\n".join(paragraphes) if paragraphes else None for section, paragraphes in resultats.items()}

# Sections cibles
sections_cibles = ["Qu'est-ce que", "Pourquoi", "Comment", "À qui s'adresse", "De quoi se compose", "Qui est", "Nos conseils", "Quel taux de nicotine", "Conservation"]

# Appliquer la fonction à chaque ligne de la DataFrame
extractions = df_unique_lien['Description'].apply(lambda x: extraire_paragraphes_multiples(x, sections_cibles))

# Ajouter les colonnes à partir des résultats
df_unique_lien['Section_Qu_est_ce_que'] = extractions.apply(lambda x: x["Qu'est-ce que"])  # produit
df_unique_lien['Section_Pourquoi'] = extractions.apply(lambda x: x["Pourquoi"])
df_unique_lien['Section_Comment'] = extractions.apply(lambda x: x["Comment"])
df_unique_lien['Section_A_Qui'] = extractions.apply(lambda x: x["À qui s'adresse"])
df_unique_lien['Section_De_Qui'] = extractions.apply(lambda x: x["De quoi se compose"])
df_unique_lien['Section_Marque'] = extractions.apply(lambda x: x["Qui est"])
df_unique_lien['Section_conseil'] = extractions.apply(lambda x: x["Nos conseils"])
df_unique_lien['Section_taux_nicotine'] = extractions.apply(lambda x: x["Quel taux de nicotine"])
df_unique_lien['Section_Conservation'] = extractions.apply(lambda x: x["Conservation"])


In [11]:
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Saveur,Section_Qu_est_ce_que,Section_Pourquoi,Section_Comment,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,fruit,10ml,50/50,...,Fruitée,"Le Ragnarok, c'est unerecette fraiche aux goût...","Simple, pratique et efficace, le e-liquide Rag...",Un e-liquide 10ml comme le Ragnarok Ultimate e...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,fruit,10ml,50/50,...,Fruitée,Le e-liquide Oni ? Unerecetteque l'on ne prése...,Pensé pour tous les vapoteurs qui débutent dan...,"Une fois le taux nicotinique sélectionné, vous...","Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,fruit,10ml,50/50,...,Fruitée,None,None,None,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,fruit,200ml,50/50,...,Fruitée,None,None,Ce e-liquide est proposé en version200mlboosté...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,fruit,10ml,50/50,...,Fruitée,\nLe e-liquide Red Astaire est unproduit prêt ...,\nLe e liquide Red Astaire 10ml est souvent re...,\nPuisqu'il est déjà prêt à être vapoté dans v...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de...,Blond Mentholé Sel de Nicotine Pur Distillat A...,<p>Voici la recette du Blond Mentholé de chez ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55765-thi...,5.90,classique,10ml,50/50,...,Mentholée,Le sel de nicotine est la forme qui se rapproc...,None,None,None,None,Le fabricant français s'est fait reconnaître p...,None,Si vous êtes un fumeur qui débute avec la ciga...,Pour une conservation optimale de votre e-liqu...
2328,https://www.aromes-et-liquides.fr/e-liquide-81...,Nominoë 50ml 814,Nominoë 50ml 814,<p>Le Nominoë est un e-liquide à booster au fo...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56383-thi...,21.90,classique,50ml (fiole de 60ml),

In [12]:
# Fonction pour extraire le premier paragraphe d'une ligne
def extraire_premier_paragraphe(description):
    # Trouver la première occurrence de <p> et </p>
    debut = description.find("<p>")
    fin = description.find("</p>")
    
    # Si les balises <p> et </p> sont trouvées, extraire le texte entre elles
    if debut != -1 and fin != -1:
        return description[debut + 3:fin].strip()  # +3 pour ignorer <p>
    else:
        return None  # Retourner None si aucune balise <p> n'est trouvée

# Appliquer la fonction à la colonne 'Description'
df_unique_lien['info'] = df_unique_lien['Description'].apply(extraire_premier_paragraphe)

création de colonne 'brand' 
- récuperer la marque du produit 
de col Description les valeurs qui se trouve entre <h2>Qui est et ?</h2>

In [13]:
import re
# Fonction pour extraire le texte entre <h2>Qui est  et  ?</h2>
def extract_text_between_tags(html_text):
    try:
        # Regex pour capturer le texte entre <h2>Qui est  et  ?</h2>
        result = re.search(r'<h2>Qui est\s*(.*?)\s*\?</h2>', html_text)
        if result:
            return result.group(1)  # Retourne le texte capturé
        return None  # Retourne None si aucun texte n'est trouvé
    except:
        return None  # Gestion des erreurs si le texte est mal formaté

# Appliquer la fonction à la colonne 'Description'
df_unique_lien['brand'] = df_unique_lien['Description'].apply(extract_text_between_tags)

In [14]:
# Récupérer toutes les valeurs de la colonne 'brand' et les mettre dans une liste
brand_list = df_unique_lien['brand'].dropna().tolist()
print(brand_list)

['Arômes et Liquides (A&L)', 'Hello Cloudy', 'Arômes et Liquides (A&L)', 'PULP', 'Alfaliquid', 'Hello Cloudy', 'Alfaliquid', "Secret's Lab", 'Alfaliquid', 'Alfaliquid', 'Hello Cloudy', 'Alfaliquid', 'Eliquid France', 'Arômes et Liquides (A&L)', 'Eliquid France', "Secret's Lab", 'Alfaliquid', 'Chuffed', 'Chuffed', "Secret's Lab", 'Curieux', 'PULP', 'T-Juice', "Secret's Lab", 'Liquideo', 'Chuffed', 'Liquideo', 'PULP', 'PULP', 'Alfaliquid', 'Eliquid France', "Secret's Lab", 'Swoke', "Secret's Lab", 'Chuffed', 'AVAP', 'PULP', 'Eliquid France', 'VDLV', 'Hello Cloudy', 'Kingston E-liquids', 'Curieux', 'Swoke', 'Swoke', 'Eliquid France', 'PULP', 'Savourea', 'Arômes et Liquides (A&L)', 'Solana', 'Swoke', 'Alfaliquid', 'Eliquid France', 'Chuffed', 'VDLV', 'Hello Cloudy', 'Arômes et Liquides (A&L)', 'Jungle Fever', 'Eliquid France', 'PULP', "Secret's Lab", 'VDLV', 'PULP', 'Hello Cloudy', 'Space\xa0Fruit', 'Eliquid France', 'Swoke', 'Curieux', "Belgi'Ohm", 'Chuffed', 'PULP', 'VDLV', 'Eliquid Fran

In [15]:
df_unique_lien['brand'].isna().sum()

73

In [16]:
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_Pourquoi,Section_Comment,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,fruit,10ml,50/50,...,"Simple, pratique et efficace, le e-liquide Rag...",Un e-liquide 10ml comme le Ragnarok Ultimate e...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,None
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,fruit,10ml,50/50,...,Pensé pour tous les vapoteurs qui débutent dan...,"Une fois le taux nicotinique sélectionné, vous...","Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...,None
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,fruit,10ml,50/50,...,None,None,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...,Arômes et Liquides (A&L)
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,fruit,200ml,50/50,...,None,Ce e-liquide est proposé en version200mlboosté...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge...",Hello Cloudy
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,fruit,10ml,50/50,...,\nLe e liquide Red Astaire 10ml est souvent re...,\nPuisqu'il est déjà prêt à être vapoté dans v...,None,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de...,Blond Mentholé Sel de Nicotine Pur Distillat A...,<p>Voici la recette du Blond Mentholé de chez ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55765-thi...,5.90,classique,10ml,50/50,...,None,None,None,None,Le fabricant français s'est fait reconnaître p...,None,Si vous êtes un fumeur qui débute avec la ciga...,Pour une conservation optimale de votre e-liqu...,Voici la recette du Blond Mentholé de chez Alf...,Alfaliquid
2328,https://www.aromes-et-liquides.fr/e-liquide-81...,Nominoë 50ml 814,Nominoë 50ml 814,<p>Le Nominoë est un e-liquide à booster au fo...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",h

a la main je corrige les brands manquants

In [17]:
# Fonction pour vérifier et mettre à jour la colonne 'brand' en fonction de la correspondance
def mettre_a_jour_brand(nom_produit, brand, valeurs_a_verifier):
    # Dictionnaire des correspondances spécifiques
    correspondances = {
        'A&L': 'A&L',
        'Puff Juice': 'Puff Juice',
        'Pulp': 'Pulp',
        'Frozen Freaks': 'Frozen Freaks',
        "Secret's Lab":"Secret's Lab",
        'Pack e-liquides du mois': 'A&L',
        "Blue Flash Cool n'fruit": 'Alfaliquid',
        "Pomme Candy 50ml Sweety Fruits": "Prestige Fruits",
        "Prestige": "Prestige Fruits"
    }

    # Vérifier les correspondances spécifiques dans 'nom_produit'
    for key, value in correspondances.items():
        if key in nom_produit:
            # Si 'brand' est None, on assigne la valeur correspondante
            if brand is None:
                return value
            else:
                return brand

    # Vérifier les autres correspondances dans 'valeurs_a_verifier'
    for valeur in valeurs_a_verifier:
        if valeur in nom_produit:
            if brand is None:
                return valeur
            else:
                return brand

    # Si aucune correspondance, retourner la valeur actuelle de 'brand'
    return brand

# Appliquer la fonction à chaque ligne du DataFrame
df_unique_lien['brand'] = df_unique_lien.apply(lambda row: mettre_a_jour_brand(row['Title'], row['brand'], brand_list), axis=1)


In [18]:
df_unique_lien['brand'].isna().sum()

0

In [19]:
# Filtrer les lignes où la colonne 'brand' est None
df_brand_manquant = df_unique_lien[df_unique_lien['brand'].isna()]

In [20]:
# Récupérer toutes les valeurs de la colonne 'brand' et les mettre dans une liste
brand_list_manquant = df_brand_manquant['Title'].dropna().tolist()
print(brand_list_manquant)

[]


In [21]:
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_Pourquoi,Section_Comment,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,fruit,10ml,50/50,...,"Simple, pratique et efficace, le e-liquide Rag...",Un e-liquide 10ml comme le Ragnarok Ultimate e...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,A&L
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,fruit,10ml,50/50,...,Pensé pour tous les vapoteurs qui débutent dan...,"Une fois le taux nicotinique sélectionné, vous...","Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...,A&L
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,fruit,10ml,50/50,...,None,None,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...,Arômes et Liquides (A&L)
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,fruit,200ml,50/50,...,None,Ce e-liquide est proposé en version200mlboosté...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge...",Hello Cloudy
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,fruit,10ml,50/50,...,\nLe e liquide Red Astaire 10ml est souvent re...,\nPuisqu'il est déjà prêt à être vapoté dans v...,None,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...,T-Juice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de...,Blond Mentholé Sel de Nicotine Pur Distillat A...,<p>Voici la recette du Blond Mentholé de chez ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55765-thi...,5.90,classique,10ml,50/50,...,None,None,None,None,Le fabricant français s'est fait reconnaître p...,None,Si vous êtes un fumeur qui débute avec la ciga...,Pour une conservation optimale de votre e-liqu...,Voici la recette du Blond Mentholé de chez Alf...,Alfaliquid
2328,https://www.aromes-et-liquides.fr/e-liquide-81...,Nominoë 50ml 814,Nominoë 50ml 814,<p>Le Nominoë est un e-liquide à booster au fo...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",

nettoyage de la col Description 

In [22]:
# Fonction pour nettoyer les balises HTML et supprimer les espaces en trop
def remove_html_tags(text):
    # Remplacer les balises <p>, </p>, <h2>, </h2>, <h3>, </h3> par des espaces
    cleaned_text = re.sub(r'<(p|h2|h3)>', ' ', text)  # Remplacer les balises d'ouverture
    cleaned_text = re.sub(r'</(p|h2|h3)>', ' ', cleaned_text)  # Remplacer les balises de fermeture
    cleaned_text = cleaned_text.replace('\n', ' ')  # Remplacer les sauts de ligne par un espace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remplacer les espaces multiples par un seul espace
    cleaned_text = cleaned_text.strip()  # Supprimer les espaces en début et fin de texte
    return cleaned_text

# Appliquer la fonction de nettoyage à la colonne "Description"
df_unique_lien['Description_cleaned'] = df_unique_lien['Description'].apply(remove_html_tags)
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_Comment,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand,Description_cleaned
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,fruit,10ml,50/50,...,Un e-liquide 10ml comme le Ragnarok Ultimate e...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,A&L,Le e-liquide Ragnarok par A&L Ultimate est un ...
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,fruit,10ml,50/50,...,"Une fois le taux nicotinique sélectionné, vous...","Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...,A&L,Le Oni Ultimate 10ml est un e-liquide pour cig...
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,fruit,10ml,50/50,...,None,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...,Arômes et Liquides (A&L),Le Fruits Rouges est un e-liquide auxsaveurs d...
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,fruit,200ml,50/50,...,Ce e-liquide est proposé en version200mlboosté...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge...",Hello Cloudy,"Le Red Light, unesaveur fraîche de baies rouge..."
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,fruit,10ml,50/50,...,\nPuisqu'il est déjà prêt à être vapoté dans v...,None,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...,T-Juice,Le Red Astaire est un e-liquide qui a rencontr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de...,Blond Mentholé Sel de Nicotine Pur Distillat A...,<p>Voici la recette du Blond Mentholé de chez ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55765-thi...,5.90,classique,10ml,50/50,...,None,None,None,Le fabricant français s'est fait reconnaître p...,None,Si vous êtes un fumeur qui débute avec la ciga...,Pour une conservation optimale de votre e-liqu...,Voici la recette du Blond Mentholé de chez Alf...,Alfaliquid,Voici la recette du Blond Mentholé de chez Alf...
2328,https://www.aromes-et-liquides.fr/e-liquide-81..

In [23]:
df_unique_lien.iloc[0]["Description"]

"<p>Le e-liquide Ragnarok par A&L Ultimate est un produitprêt à l'emploipour cigarette électronique. Dans sa fiole 10ml, vous retrouverez des saveurs de fruits rouges, accompagnées d'un goût frais de cassis.</p>\n<p>Idéal pour débuter, ce e-liquide vous est proposé en différents taux de nicotine au choix : 0, 3, 6, ou 12mg/ml.</p>\n<p></p>\n<h2>Qu'est-ce que le Ragnarok Ultimate\xa0?</h2>\n<p>Le Ragnarok, c'est unerecette fraiche aux goûts de fruitsdehaute qualité française, signée Arômes et Liquides (A&L). Ici, vous en profiterez dans sa version e-liquide prêt à vaper.</p>\n<p>Conforme à la loi TPD, le e liquide Ragnarok est présenté en fiole PET de 10ml. Pourquoi\xa0? Afin de vous assurer unproduit déjà nicotiné à votre convenance!</p>\n<p></p>\n<h2>Pourquoi choisir le Ragnarok en e-liquide 10ml\xa0?</h2>\n<p>Simple, pratique et efficace, le e-liquide Ragnarok en 10ml permet de démarrer sans attendre votre séance de vapotage. Déjà minutieusement travaillé, dosé et nicotiné, il vous s

### 4. Création de la colonne saveur

In [24]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy

In [25]:
# Charger les ressources nécessaires
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
nlp = spacy.load('fr_core_news_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\antoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
c:\Users\antoa\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Listes des saveurs
fruits = [
    "fraise", "framboise", "cerise", "myrtille", "cassis", "mûre", "abricot", "pêche", "nectarine",
    "poire", "pomme", "orange", "mandarine", "citron", "citron vert", "pamplemousse", "raisin", 
    "kiwi", "mangue", "ananas", "papaye", "fruit de la passion", "grenade", "figue", "datte",
    "banane", "litchi", "noix de coco", "melon", "pastèque", "goji", "groseille", "prune", 
    "quince", "kumquat", "yuzu", "bergamote", "lime", "fruits rouges", "fruit rouge", "ananas", "fruit de la passion",
    "pastèque", "pèche", "tuttifruiti", "tutti fruiti", "dagrumes", "pitaya", "fruits des bois",
    "fruit du dragon", "fruit des bois", "baies noires", "baie noire", "baies", "baie", "fruits du dragon",
    "fruit du dragon", "lemon", "corossol", "fruits tropicaux", "fruit tropical", "baies rouges", "cactus",
    "agrumes", "clémentine", "clementine", 

]

desserts = [
    "chocolat", "caramel", "vanille", "crème brûlée", "praline", "tarte", "tiramisu", "éclair",
    "macaron", "brownie", "cheesecake", "gâteau", "sablé", "biscuit", "pancake", "crêpe",
    "beignet", "canelé", "cupcake", "muffin", "gaufre", "mousse au chocolat", "flan",
    "pudding", "sorbet", "glace", "chantilly", "nougat", "pain d'épices", "fondant",
    "coulant", "madeleine", "crème anglaise", "riz au lait", "forêt noire", "mille-feuille",
    "clafoutis", "parfait", "génoise", "tartiflette sucrée", "bubblegum", "vanilla", "bonbons", "bonbon",
    "chewinggum", "barbe à papa", "cookie", "cookies", "cookie vanillé", "cookie chocolat", "chocolat", "lait",
    "céréales",  "céréale", "crème pralinée", "donut", "cornflakes", "custard", "crème custard vanillée", "croissant",
    "bubble gum"
]

boissons = [
    "café", "thé", "latte", "cappuccino", "expresso", "mocha", "chocolat chaud",
    "limonade", "soda", "cola", "jus d'orange", "jus de pomme", "cocktail",
    "punch", "smoothie", "milkshake", "bubble tea", "thé glacé", "infusion",
    "vin", "champagne", "bière", "cidre", "whisky", "rhum", "vodka",
    "liqueur", "menthe à l'eau", "grenadine", "kefir", "kombucha", "cognaq", "mojito"
]

gouts_classiques = [
    "menthe", "tabac blond", "tabac brun", "cannelle", "anis", "réglisse",
    "miel", "noisette", "amande", "cacahuète", "noix", "sucre roux",
    "épices", "gingembre", "clou de girofle", "poivre", "eucalyptus",
    "bois de santal", "vanille bourbon", "feuille de tabac", "thym",
    "herbes", "menthol", "cèdre", "vanille sucrée", "caramel beurre salé",
    "beurre de cacahuète", "popcorn", "toast grillé", "châtaigne",
    "savon floral", "lavande", "rose", "violette", "tabac", "british tabac", 
    "american mix tabac", "tabac le classic"
]

gouts_tabac = [
    # Tabacs classiques
    "tabac blond", "tabac classique blond", "tabac brun","tabac classique brun", "tabac américain", "tabac burley", 
    "tabac virginia", "tabac oriental", "tabac cubain",

    # Tabacs aromatisés
    "tabac vanille", "tabac caramel", "tabac miel", "tabac chocolat",
    "tabac café", "tabac menthe", "tabac noisette", "tabac épicé",
    "tabac bourbon", "tabac rhum", "tabac whisky",

    # Tabacs gourmands
    "tabac crème", "tabac praliné", "tabac biscuit", "tabac noix de coco",
    "tabac custard", "tabac tiramisu", "tabac cacahuète", "tabac caramel beurre salé",

    # Tabacs corsés ou boisés
    "tabac corsé", "tabac fumé", "tabac boisé", "tabac grillé",
    "tabac cendré", "tabac sec", "tabac intense", "tabac terreux",

    # Tabacs fruités
    "tabac pomme", "tabac cerise", "tabac fruits rouges", "tabac raisin",
    "tabac agrumes", "tabac figue", "tabac prune",

    # Tabacs mentholés
    "tabac menthol", "tabac menthe glaciale", "tabac menthe verte",

    # Tabacs exotiques
    "tabac exotique", "tabac épices orientales", "tabac poivré", 
    "tabac réglisse", "tabac anis", "tabac gingembre",

    # Mélanges spécifiques
    "tabac américain gourmand", "tabac oriental sucré", "tabac cubain épicé"
]

# Combiner les deux dictionnaires pour la recherche
tous_les_mots = set(fruits + desserts + boissons + gouts_classiques + gouts_tabac)


Création de col saveur 

# Fonction pour trouver les mots dans chaque phrase
def trouver_mots_par_phrase(description, mots):
    description = description.lower()

    # Traiter le texte avec spaCy
    doc = nlp(description)
    
    # Initialiser une liste pour les mots trouvés
    mots_trouves = []
    
    # Parcourir chaque phrase dans le document
    for phrase in doc.sents:
        # Chercher si un des mots ou expressions est dans la phrase
        for mot in mots:
            if mot in phrase.text:
                mots_trouves.append(mot)
    # Si aucun mot n'a été trouvé, retourner None
    if not mots_trouves:
        return None
    # Retourner la liste des mots trouvés sans doublons
    return ", ".join(set(mots_trouves))

Fonction pour trouver des mots spécifiques de la dictionnaire avec les saveurs possible dans la description et le titre

In [27]:
def trouver_mots_par_phrase(description, mots, title):
    # Traiter le texte avec spaCy
    doc = nlp(description)
    
    # Initialiser une liste pour les mots trouvés
    mots_trouves = []
    
    # Parcourir chaque phrase dans le document
    for phrase in doc.sents:
        # Chercher si un des mots ou expressions est dans la phrase
        for mot in mots:
            if mot in phrase.text:
                mots_trouves.append(mot)
    
    # Vérifier si les mots sont dans le Title
    for mot in mots:
        if mot in title:
            mots_trouves.append(mot)
    
    # Si aucun mot n'a été trouvé, retourner None
    if not mots_trouves:
        return None
    
    # Retourner la liste des mots trouvés sans doublons
    return ", ".join(set(mots_trouves))

In [28]:

df_unique_lien['saveur'] = df_unique_lien.apply(
    lambda x: trouver_mots_par_phrase(x['Description_cleaned'], tous_les_mots, x['Title']), axis=1
)

In [30]:
def trouver_mots_par_phrase(description, mots, title):
    # Normaliser la casse pour une meilleure détection
    description = description.lower() if description else ""
    title = title.lower() if title else ""
    mots = [mot.lower() for mot in mots]

    # Traiter le texte avec spaCy
    doc = nlp(description)
    
    # Initialiser une liste pour les mots trouvés
    mots_trouves = set()

    # Vérifier chaque phrase du texte
    for phrase in doc.sents:
        for mot in mots:
            if mot in phrase.text:
                mots_trouves.add(mot)
    
    # Vérifier les mots dans le titre
    for mot in mots:
        if mot in title:
            mots_trouves.add(mot)
    
    # Retourner la liste des mots trouvés ou None si aucun mot n'a été trouvé
    return ", ".join(mots_trouves) if mots_trouves else None

In [31]:
df_unique_lien['saveur'] = df_unique_lien.apply(
    lambda x: trouver_mots_par_phrase(x['Description_cleaned'], tous_les_mots, x['Title']), axis=1
)

# Appliquer la fonction pour chaque ligne de la colonne info_produit
df_unique_lien['saveur'] = df_unique_lien['Description_cleaned'].apply(lambda x: trouver_mots_par_phrase(x, tous_les_mots))

In [32]:
df_unique_lien['saveur'].isna().sum()

2

In [33]:
df_unique_lien[df_unique_lien['saveur'].isna()]

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand,Description_cleaned,saveur
363,https://www.aromes-et-liquides.fr/e-liquide-ma...,California Marie Jeanne,California Marie Jeanne,<p>En vous proposant son liquide au CBD Califo...,"{'Contenance': '10ml', 'Taux PG/VG': '65/35', ...",https://assets.aromes-et-liquides.fr/40756-thi...,16.9,fruit,10ml,65/35,...,None,None,Découvrez un fabricant français spécialisé dan...,None,None,Pour une conservation optimale de votre e-liqu...,En vous proposant son liquide au CBD Californi...,Marie Jeanne,En vous proposant son liquide au CBD Californi...,None
2188,https://www.aromes-et-liquides.fr/e-liquide-pu...,Classic Tennessee Blend PULP,Classic Tennessee Blend PULP,<p>Retrouvez le goût d'unclassic blond américa...,"{'Contenance': '10ml', 'Taux PG/VG': '70/30', ...",https://assets.aromes-et-liquides.fr/54910-thi...,5.9,classique,10ml,70/30,...,None,None,"C'est dans leur laboratoire français, et plus ...",L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Retrouvez le goût d'unclassic blond américaind...,PULP,Retrouvez le goût d'unclassic blond américaind...,None


In [35]:
df_unique_lien['saveur'].unique

<bound method Series.unique of 0                    cassis, lime, fruits rouges, parfait
1                      citron vert, lime, parfait, citron
2          baies, baie, fruits rouges, lime, baies rouges
3                         lime, baie, baies, baies rouges
4       fruits rouges, raisin, menthol, anis, lime, me...
                              ...                        
2327                   tabac, menthol, anis, lime, menthe
2328                               caramel, lime, biscuit
2329                                          anis, tabac
2330                                    anis, lime, tabac
2331                                    anis, lime, tabac
Name: saveur, Length: 1533, dtype: object>

In [36]:
import spacy
import re
from spacy.lang.fr.stop_words import STOP_WORDS
from unidecode import unidecode

# Charger le modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")

def nettoyer_description(description):
    # Vérifier si le texte est vide
    if not description:
        return None

    # 1. Convertir en minuscules
    description = description.lower()

    # 2. Supprimer les accents
    description = unidecode(description)

    # 3. Supprimer les caractères spéciaux et ponctuations
    description = re.sub(r'[^\w\s]', ' ', description)

    # 4. Traiter avec spaCy
    doc = nlp(description)

    # 5. Supprimer les mots vides et faire de la lemmatisation
    mots_nettoyes = [
        token.lemma_
        for token in doc
        if token.lemma_ not in STOP_WORDS and not token.is_punct and len(token) > 1
    ]

    # 6. Rejoindre les mots nettoyés en une seule chaîne
    return " ".join(mots_nettoyes)

# Appliquer la fonction à la colonne 'Description_cleaned'
df_unique_lien['Description_cleaned_normalized'] = df_unique_lien['Description_cleaned'].apply(nettoyer_description)


In [37]:
df_unique_lien


,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand,Description_cleaned,saveur,Description_cleaned_normalized
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,fruit,10ml,50/50,...,Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,A&L,Le e-liquide Ragnarok par A&L Ultimate est un ...,"cassis, lime, fruits rouges, parfait",liquide ragnarok ultimate produitpret emploipo...
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,fruit,10ml,50/50,...,Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...,A&L,Le Oni Ultimate 10ml est un e-liquide pour cig...,"citron vert, lime, parfait, citron",oni ultimat 10ml liquide cigarette electroniqu...
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,fruit,10ml,50/50,...,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...,Arômes et Liquides (A&L),Le Fruits Rouges est un e-liquide auxsaveurs d...,"baies, baie, fruits rouges, lime, baies rouges",fruit rouge liquide auxsaveur baie rougespour ...
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,fruit,200ml,50/50,...,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge...",Hello Cloudy,"Le Red Light, unesaveur fraîche de baies rouge...","lime, baie, baies, baies rouges",red light unesaveur fraich baie rouge recette ...
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,fruit,10ml,50/50,...,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...,T-Juice,Le Red Astaire est un e-liquide qui a rencontr...,"fruits rouges, raisin, menthol, anis, lime, me...",red astair liquide rencontre enorme succe arri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de...,Blond Mentholé Sel de Nicotine Pur Distillat A...,<p>Voici la recette du Blond Mentholé de chez ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55765-thi...,5.90,classique,10ml,50/50,...,None,Le fabricant français s'est fait reconnaître p...,None,Si vous êtes un fumeur qui débute avec la ciga...,Pour une conservation optimale de votre e-liqu...,Voici la recette du Blond Menth

In [38]:
df_unique_lien.to_csv('csv_clean/A&L.csv', index=False)